In [ ]:
import torch 
from train import trainAR
from models import *
from matplotlib import pyplot as plt
import build
from tqdm.notebook import tqdm
import pytorch_lightning as pl
import torch.nn.functional as F

In [ ]:
import build
loader = build.dataloader(data='dataset_v6', part=0.1, batch_size=10, shuffle=True)
print(len(loader))

In [ ]:
x, y = next(iter(loader))
print(x.shape, y.shape)

In [ ]:
print(x.min(), x.max())

In [ ]:
width = 10
premodel = Sequential(
    ConvBlock(256, width, 1, shift=1),
    Res(GatedConvBlock(width, width, 2)),
    Res(GatedConvBlock(width, width, 4)),
    Res(GatedConvBlock(width, width, 8)),
    Res(GatedConvBlock(width, width, 16)),
    Res(GatedConvBlock(width, width, 32)),
    Res(GatedConvBlock(width, width, 64)),
    GatedConvBlock(width, 256, 128),
    torch.nn.Tanh(),
)

In [ ]:
class DaNet(pl.LightningModule):
    def __init__(self, model_loader, noise=0.0):
        super().__init__()
        self.save_hyperparameters()
        self.model = eval(model_loader)
    
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, aux = batch
        x += torch.randn_like(x) * self.hparams.noise
        x_hat = self.forward(x)
        loss = F.mse_loss(x_hat, x)
        self.log("train_loss", loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters())
        return optimizer

In [ ]:
model = DaNet(module_description(premodel), noise=0.3)
trainer = pl.Trainer(log_every_n_steps=5)
trainer.fit(model, train_dataloaders=loader)

In [ ]:
print(model)

In [ ]:
# trainAR(
#     trial=None,
#     device='cpu',
#     loader=loader,
#     noise=0.1,
#     model=model,
#     optim=torch.optim.Adam(model.parameters()),
#     sched=None,
#     criterion=build.criterion(),
#     logger=build.logger(console=True, save_rate=1e-3, sample_rate=0.5),
#     epochs=10
# )